In [0]:
# Import psycopg to send data to Postgresql
import psycopg2

In [0]:
%run ./Config

In [0]:
# Creating a function for bulk insertion in SQL PostgreSQL
def bulk_insert(table, records):
    try:
        # Create a connection to the database
        connection = psycopg2.connect(CONNECTION)

        cursor = connection.cursor()
        
        if table == 'monthly':
            sql_insert_query = f""" INSERT INTO {table} (ticker, year, month, open, high, low, close, volume)
                                    VALUES (%s,%s,%s,%s,%s,%s,%s,%s) """
        elif table == 'weekly':
            sql_insert_query = f""" INSERT INTO {table} (ticker, year, week, open, high, low, close, volume)
                                    VALUES (%s,%s,%s,%s,%s,%s,%s,%s) """
        elif table == 'indicators':
            sql_insert_query = f""" INSERT INTO {table} (indicator, value, date)
                                    VALUES (%s,%s,%s) """
        elif table == 'stocks':
            sql_insert_query = f""" INSERT INTO {table} (date, ticker, open, high, low, close, volume, RSI, ma7, ma23, ma180)
                                    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) """
        else:
            sql_insert_query = f""" INSERT INTO {table} (date, ticker, open, high, low, close, volume, RSI)
                                    VALUES (%s,%s,%s,%s,%s,%s,%s,%s) """

        # executemany() to insert multiple rows
        result = cursor.executemany(sql_insert_query, records)
        connection.commit()
        print(cursor.rowcount, "Record inserted successfully into mobile table")

    except (Exception, psycopg2.Error) as error:
        print(f"Failed inserting record into the table {error}")

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")


# Dataframe to list of tuples for SQL INSERT
def df_to_tuple(df):
    "Convert a Spark dataframe to a list of tuples"
    # convert dataframe to rdd 
    rdd = df.rdd   
    # convert rdd to tuple 
    to_tuple = rdd.map(tuple).collect()  

    return to_tuple


In [0]:
# Function to create table in PostgreSQL
def create_table(name, sql):
    "This function opens a connection with PostgreSQL and creates a table based on the sql command provided"
    
    # Create a connection and cursor
    conn = psycopg2.connect(CONNECTION)
    cursor = conn.cursor()

    # Execture SQL Commmands
    cursor.execute(sql)
    
    # Commit and close connection
    conn.commit()
    cursor.close()
    conn.close()
    
    return "Table created"


In [0]:
# SQL command 1 for Monthly Data Table
sql = f'''
    CREATE TABLE monthly (
    ticker VARCHAR(255),
    year INT,
    month INT,
    open DECIMAL(10,2),
    high DECIMAL(10,2),
    low DECIMAL(10,2), 
    close DECIMAL(10,2), 
    volume INT
   );
'''

# Creating the table
create_table(name='monthly', sql= sql)


In [0]:
# Loading monthly stock dataset
dtf = spark.read.parquet('/FileStore/tables/gold/monthly_stocks')

# Converto to list of tuple for SQL loading
records_to_insert = df_to_tuple(dtf)

# Load to PostgreSQL
bulk_insert(table='monthly', records= records_to_insert)

305 Record inserted successfully into mobile table
PostgreSQL connection is closed


In [0]:
# Creating a table in PostgreSQL
# SQL command 2 for Weekly Data Table
sql2 = f'''
    CREATE TABLE weekly (
    ticker VARCHAR(255),
    year INT,
    week INT,
    open DECIMAL(10,2),
    high DECIMAL(10,2),
    low DECIMAL(10,2), 
    close DECIMAL(10,2), 
    volume INT
   );
'''

# Creating the table
create_table(name='weekly', sql= sql2)

In [0]:
# Loading monthly stock dataset
dtf = spark.read.parquet('/FileStore/tables/gold/weekly_stocks')

# Converto to list of tuple for SQL loading
records_to_insert = df_to_tuple(dtf)

# Load to PostgreSQL
bulk_insert(table='weekly', records= records_to_insert)

1305 Record inserted successfully into mobile table
PostgreSQL connection is closed


In [0]:
# Creating a table in PostgreSQL
# SQL command 3 for stocks Data Table
sql3 = f'''
    CREATE TABLE stocks (
    date DATE,
    ticker VARCHAR(255),
    open DECIMAL(10,2),
    high DECIMAL(10,2),
    low DECIMAL(10,2), 
    close DECIMAL(10,2), 
    volume INT,
    RSI DECIMAL(10,2),
    ma7 DECIMAL(10,2),
    ma23 DECIMAL(10,2),
    ma180 DECIMAL(10,2)
   );
'''

# Creating the table
create_table(name='stocks', sql= sql3)

'Table created'

In [0]:
# Loading monthly stock dataset
dtf = spark.read.parquet('/FileStore/tables/silver/stocks')

# Converto to list of tuple for SQL loading
records_to_insert = df_to_tuple(dtf)

# Load to PostgreSQL
bulk_insert(table='stocks', records= records_to_insert)

In [0]:
# Creating a table in PostgreSQL
# SQL command 3 for economic indicators Data Table
sql4 = f'''
    CREATE TABLE indicators (
    indicator VARCHAR(255),
    value DECIMAL(10,2),
    date DATE
   );
'''

# Creating the table
create_table(name='indicators', sql= sql4)

In [0]:
# Loading monthly stock dataset
dtf = spark.read.parquet('/FileStore/tables/silver/indicators')

# Converto to list of tuple for SQL loading
records_to_insert = df_to_tuple(dtf)

# Load to PostgreSQL
bulk_insert(table='indicators', records= records_to_insert)

82 Record inserted successfully into mobile table
PostgreSQL connection is closed


In [0]:
# Creating a table in PostgreSQL
# SQL command 3 for Telco Index Data Table
sql5 = f'''
    CREATE TABLE telco_idx (
    date DATE,
    ticker VARCHAR(255),
    open DECIMAL(10,2),
    high DECIMAL(10,2),
    low DECIMAL(10,2), 
    close DECIMAL(10,2), 
    volume BIGINT,
    RSI DECIMAL(10,2)
   );
'''

# Creating the table
create_table(name='telco_idx', sql= sql5)

'Table created'

In [0]:
# Loading monthly stock dataset
dtf = spark.read.parquet('/FileStore/tables/silver/DJUSTL')

# Converto to list of tuple for SQL loading
records_to_insert = df_to_tuple(dtf)

# Load to PostgreSQL
bulk_insert(table='telco_idx', records= records_to_insert)

949 Record inserted successfully into mobile table
PostgreSQL connection is closed
